# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingenieria de Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Proyecto Final**: Proyecto Team Bellaco

**Fecha**:13/05/2025

**Nombre del Estudiante**:David Abraham Naranjo Salgado, Benjamin Leonardo Zarate Solano y Angel David Cortes Pacheco

**Profesor**: Pablo Camarillo Ramirez

# Introduction and Problem Definition

En este proyecto final de Procesamiento de Datos Masivos, desarrollamos una aplicación de análisis y recomendación de contenido en una plataforma de video streaming, inspirada en servicios como Netflix o YouTube. El objetivo fue simular un entorno real donde múltiples tipos de datos (usuarios, películas y series) se generan y procesan en tiempo real, permitiendo construir un sistema de recomendaciones personalizadas basado en los gustos y afinidades del usuario.

Para lograrlo, se utilizaron las siguientes tecnologías clave:

- **Kafka** para la generación y transmisión de datos en tiempo real, con tres productores simulando usuarios, series y películas.
- **PySpark Structured Streaming** para el consumo de datos desde Kafka y su almacenamiento en archivos Parquet, formando un mini Data Lake.
- **ALS ()** para construir un modelo de recomendación colaborativo basado en coincidencias de preferencias entre usuarios y contenido.
- **Power BI** como herramienta de visualización de las recomendaciones generadas.

# System Architecture

La arquitectura de nuestra aplicación se basa en un flujo de procesamiento de datos en tiempo real, desde la generación de paquetes hasta la generacion de recomendaciones. Se compone de los siguientes componentes principales:

1. **Producers Kafka**  
   Tres productores independientes generan datos simulados:
   - `Producer A`: genera información de usuarios (id, type, timestamp, nombre_usuario, fecha_suscripcion, generos, directores, actores, clasificacion, decadas_favoritas).
   - `Producer B`: genera información de películas (id, type, timestamp, titulo, generos, directores, actores, clasificacion, decada_lanzamiento).
   - `Producer C`: genera información de series (id, type, timestamp, titulo, generos, directores, actores, clasificacion, numero_temporadas, numero_episodios).

2. **Apache Kafka**  
   Cada productor publica en un tópico diferente dentro del clúster Kafka, permitiendo que los datos fluyan en paralelo.

3. **PySpark Structured Streaming**  
   Se utiliza para leer en tiempo real los mensajes de Kafka, aplicar un esquema estructurado, y almacenar los datos en formato Parquet, organizados por tópico, creando así un pequeño **Data Lake**.

4. **Modelo de Recomendación (ALS)**  
   A partir de los datos almacenados, se construye una tabla de afinidades con columnas como `userId - contentId - rating`, y se entrena un modelo de recomendaciones basado en ALS (Alternating Least Squares), propio de filtrado colaborativo.

5. **Exportación CSV + Power BI**  
   Las recomendaciones generadas por el modelo se exportan como archivo CSV, que luego se carga en Power BI para mostrar insights y recomendaciones personalizadas por usuario.

---

### Diagrama de Arquitectura

<center> <img src="../img/Arquitecture.png" alt="Arquitecture" width="250" height="400"> </center>

# 5Vs Justification

A continuación se explica cómo nuestra aplicación cumple con las 5Vs del Big Data:

## • Volume (Volumen)

El sistema maneja grandes volúmenes de datos al simular una plataforma de streaming en la que cada productor genera mensajes ricos en contenido (listas de géneros, directores, actores, etc.). 

Tamaño promedio de cada registro en bytes:

| Time Period              | Data Processed |
|--------------------------|----------------|
| 1 Second                 | 500 KB         |
| 1 Minute (60 Seconds)    | 30 MB          |
| 1 Hour (3,600 Seconds)   | 1.8 GB         |
| 1 Day (86,400 Seconds)   | 43.2 GB        |
| 1 Year (31.5M Seconds)   | 15.7 TB        |

Con 3 productores enviando datos simultáneamente, el sistema puede alcanzar varios GB de datos procesados en pocas horas.

---

## • Velocity (Velocidad)

El sistema fue diseñado para funcionar en tiempo real. Utilizamos **Kafka** para ingesta continua y **PySpark Structured Streaming** para procesamiento inmediato. La métrica `processedRowsPerSecond`, obtenida desde los `QueryListeners` de Spark, permite cuantificar el rendimiento del sistema en producción.

---

## • Variety (Variedad)

Se trabajan tres tipos distintos de datos:
- **Usuarios** (`type = "user"`): con campos como `nombre_usuario`, `generos`, `decadas_favoritas`, etc.
- **Películas** (`type = "movie"`)
- **Series** (`type = "series"`)

Cada uno tiene estructuras diferentes y listas de preferencias o características únicas. El esquema utilizado en PySpark refleja esta variedad:

```python
StructType([
    StructField("id", StringType(), True),
    StructField("type", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("titulo", StringType(), True),
    StructField("nombre_usuario", StringType(), True),
    StructField("fecha_suscripcion", StringType(), True),
    StructField("generos", ArrayType(StringType()), True),
    StructField("directores", ArrayType(StringType()), True),
    StructField("actores", ArrayType(StringType()), True),
    StructField("clasificacion", StringType(), True),
    StructField("decada_lanzamiento", StringType(), True),
    StructField("numero_temporadas", StringType(), True),
    StructField("numero_episodios", StringType(), True),
    StructField("decadas_favoritas", ArrayType(StringType()), True),
])
```

## • Variety (Variedad)

El uso de esquemas en PySpark permite asegurar que cada mensaje recibido desde Kafka cumpla con una estructura válida. Además, se filtran por campo type para evitar inconsistencias al procesar diferentes entidades (usuario, película o serie).

## • Value (Valor)

El valor de esta solución se refleja en las recomendaciones generadas por el modelo de Machine Learning. Estas recomendaciones personalizadas pueden visualizarse en Power BI y tienen aplicación directa para mejorar la experiencia del usuario en plataformas reales de streaming, impulsando el engagement y la fidelización.

## • Code Quality (Calidad del Código)

La calidad del código fue una prioridad durante el desarrollo del proyecto. Se abordaron los siguientes aspectos clave:

### • Functionality:
    La aplicación cumple con los requisitos planteados, integrando correctamente el manejo de datos en tiempo real mediante Kafka y Structured Streaming, así como el procesamiento por lotes para entrenamiento de modelos.

### • Code Structure and Organization:
    Todo el código fue organizado en módulos separados por responsabilidad (producers, streaming, model). Se utilizaron funciones auxiliares dentro de lib/ para mantener la limpieza del código. Se agregaron comentarios explicativos y se modularizó cada componente para facilitar mantenimiento y pruebas.

### • Efficiency:
    Las transformaciones fueron agrupadas antes de cualquier acción costosa en Spark. El uso de Structured Streaming permite procesar datos de forma eficiente y escalable. Las recomendaciones se filtran y explotan usando joins optimizados y acciones distribuidas.

# Implementation Details

La solución fue desarrollada utilizando un enfoque modular, separando cada componente clave en carpetas específicas. A continuación se describe cada parte de la implementación.

---

## Estructura del Proyecto

TeamBellaco/

    structured_streaming_kafka.ipynb
    recommendation_model.ipynb

    recommendations/
    final_recommendations.csv/

    lib/ 
    data_gen.py
    kafka_producer-a.ipynb
    kafka_producer-b.ipynb
    kafka_producer-c.ipynb
    rating_gen.py
    title_gen.py

    /home/jovyan/data/
    parquet_output/

---

## 1. Generación de Datos (Kafka Producers)

Se diseñaron tres producers en formato notebook (`ipynb`) que utilizan funciones del módulo `lib/raiting_gen.py`. Cada productor genera mensajes JSON en tiempo real con estructura diferente:

- **Producer A**: usuarios (`type = "user"`)
- **Producer B**: películas (`type = "movie"`)
- **Producer C**: series (`type = "series"`)

Cada mensaje es enviado a un tópico Kafka distinto: `producer1_topic`, `producer2_topic`, y `producer3_topic`.

---

## 2. Procesamiento en Tiempo Real (PySpark Structured Streaming)

El script `structured_streaming_kafka.ipynb` ubicado en `TeamBellaco/` consume datos desde los tres tópicos Kafka utilizando PySpark Structured Streaming. El esquema es explícitamente definido para validar los campos del JSON.

Los datos se transforman y almacenan como archivos Parquet en la carpeta `"/home/jovyan/data`, lo cual actúa como nuestro **mini Data Lake**.

---

## 3. Preparación y Entrenamiento del Modelo de Recomendación (ALS)

En el notebook `TeamBellaco/recommendation_model.ipynb` se realiza lo siguiente:

1. Se leen los Parquet files generados por cada productor.
2. Se separan los registros por tipo: usuarios vs. contenidos.
3. Se generan pares `userId - contentId` según coincidencias en preferencias (géneros, actores, etc.).
4. Se simula un valor de `rating` basado en afinidad entre usuario y contenido.
5. Se entrena un modelo de **ALS (Alternating Least Squares)** de PySpark MLlib.
6. Se generan recomendaciones personalizadas por usuario.

---

## 4. Exportación y Visualización (Power BI)

Las recomendaciones generadas se exportan como un archivo CSV en la carpeta `recommendations/final_recommendations.csv/`. Este archivo contiene:

- `userId`
- `recommended_content`
- `titulo`
- `predicted_rating`

Este CSV se utiliza para construir un dashboard de Power BI con visualizaciones interactivas.

---

## Tecnologías Utilizadas

- **Apache Kafka**: plataforma de streaming de eventos para datos en tiempo real.
- **PySpark Structured Streaming**: procesamiento distribuido de datos con esquemas definidos.
- **PySpark MLlib**: motor de aprendizaje automático distribuido.
- **Power BI**: visualización de datos y generación de dashboards.
- **Docker (opcional)**: entorno de ejecución para Kafka/ZooKeeper y Spark.



# Results and Evaluation

A continuación se presentan los resultados obtenidos tras entrenar el modelo de recomendación y generar las predicciones para cada usuario.

---

## Evaluación del Modelo

Se utilizó el modelo **ALS (Alternating Least Squares)** para realizar recomendaciones de contenido basado en afinidad entre usuarios y películas/series. Para evaluar su rendimiento, se utilizó la métrica **RMSE (Root Mean Squared Error)** sobre el conjunto de prueba (`test`).

```python
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("RMSE:", rmse)
```
RMSE del modelo: 3.6799

## Recomendaciones Generadas

<center> <img src="../img/Arquitecture.png" alt="Arquitecture" width="400" height="400"> </center>

## Dashboard en Power BI

<center> <img src="../img/Arquitecture.png" alt="Arquitecture" width="600" height="400"> </center>

# Conclusion

# 6. Conclusiones

Este proyecto permitió simular y construir una solución completa de análisis y recomendación de contenido en tiempo real, utilizando tecnologías clave en el ecosistema de Big Data. A lo largo del desarrollo, se integraron correctamente componentes de ingesta, procesamiento distribuido, modelado de datos y visualización.

- El uso de Apache Kafka permitió simular flujos de datos en tiempo real desde múltiples fuentes con distintos esquemas.
- La implementación de **PySpark Structured Streaming permitió el procesamiento en paralelo de grandes volúmenes de datos con validación estructurada.
- La creación de un mini Data Lake en formato Parquet facilitó la persistencia y análisis posterior de los datos generados.
- El modelo ALS logró generar recomendaciones personalizadas basadas en coincidencias entre usuarios y características del contenido, con un RMSE aceptable en los resultados de prueba.
- La visualización con Power BI demostró cómo los datos generados pueden transformarse en valor real para el usuario final.

### Retos identificados:
- Ajustar el esquema para cubrir diferentes tipos de contenido implicó validaciones adicionales.
- Generar ratings realistas requirió diseñar una lógica de afinidad personalizada.
- Asegurar la eficiencia de Spark al trabajar con datos en streaming y batch simultáneamente.

### Posibles mejoras:
- Incluir métricas de popularidad o visualizaciones históricas.
- Incorporar feedback real del usuario (implícito o explícito) como refuerzo del modelo.
- Escalar el sistema a múltiples nodos Spark y Kafka distribuidos.

Este proyecto no solo consolida los conocimientos adquiridos en el curso, sino que también demuestra la aplicabilidad real de las tecnologías de Big Data en soluciones de recomendación personalizadas.
